# 1. Connect kaggle API and get data

In [1]:
import os
import shutil

# Đường dẫn đến thư mục cần xóa
working_directory = "/kaggle/working/"

# Hàm xóa tất cả các tệp và thư mục trong đường dẫn cụ thể
def delete_contents(directory):
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        if os.path.isfile(item_path):
            os.remove(item_path)
            print(f"Deleted file: {item_path}")
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)
            print(f"Deleted directory: {item_path}")

# Gọi hàm để xóa nội dung trong thư mục /kaggle/working/
delete_contents(working_directory)

print("All files and folders in /kaggle/working/ have been deleted.")

Deleted file: /kaggle/working/__notebook__.ipynb
All files and folders in /kaggle/working/ have been deleted.


## Load data raw

# 2. Swin Transformers

## 2.1. Setup

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split

## 2.2. Normalization
**Với:**
- XY1 defocused_blur
- XY2 motion_blur
- XY3 noise
- XY4 rain

### 2.2.1. Preparation data

In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Kích thước ảnh chuẩn
target_size = (128, 128)

def process_image(image_path):
    try:
        image = cv2.imread(image_path)

        # Kiểm tra xem ảnh có được đọc thành công hay không
        if image is None:
            print(f"Không thể đọc ảnh từ đường dẫn: {image_path}")
            return None

        # Chuyển đổi sang dạng RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Chuẩn hóa kích thước
        image = cv2.resize(image, target_size)

        # Chuẩn hóa về khoảng [0, 1]
        image = image / 255.0

        return image
    except Exception as e:
        print(f"Lỗi khi xử lý ảnh từ đường dẫn {image_path}: {e}")
        return None
    
# Hàm để tạo dữ liệu từ một thư mục và chỉ chọn các file có đuôi .jpg
def create_data_with_jpg(data_dir, label):
    X = []
    Y = []

    for filename in os.listdir(data_dir):
        if filename.lower().endswith('.jpg'):
            img_path = os.path.join(data_dir, filename)

            # Xử lý ảnh và thêm vào danh sách
            image_data = process_image(img_path)
            
            # Kiểm tra xem ảnh có được xử lý thành công hay không
            if image_data is not None:
                X.append(image_data)

                # Thêm nhãn vào danh sách
                Y.append(label)

    # Chuyển danh sách thành numpy arrays
    X = np.array(X)
    Y = np.array(Y)

    # In thông tin
    print(f'X shape: {X.shape} - Y shape: {Y.shape}')

    return X, Y

# Tạo dữ liệu cho từng nhóm ảnh chỉ với các file có đuôi .jpg và có thể đọc được
X1, Y1 = create_data_with_jpg('/kaggle/input/dataset-collection-defocus-blur', 1)
X2, Y2 = create_data_with_jpg('/kaggle/input/dataset-collection-motion-blur', 2)
X3, Y3 = create_data_with_jpg('/kaggle/input/dataset-collection-noise', 3)
X4, Y4 = create_data_with_jpg('/kaggle/input/dataset-collection-rain', 4)

Không thể đọc ảnh từ đường dẫn: /kaggle/input/dataset-collection-defocus-blur/0313_defocus-blur.jpg


libpng error: IDAT: CRC error


Không thể đọc ảnh từ đường dẫn: /kaggle/input/dataset-collection-defocus-blur/1024_defocus-blur.jpg


X shape: (1049, 128, 128, 3) - Y shape: (1049,)


Không thể đọc ảnh từ đường dẫn: /kaggle/input/dataset-collection-motion-blur/0707_motion-blur.jpg


X shape: (7199, 128, 128, 3) - Y shape: (7199,)


Không thể đọc ảnh từ đường dẫn: /kaggle/input/dataset-collection-noise/2779_noise.jpg


X shape: (3160, 128, 128, 3) - Y shape: (3160,)


X shape: (13711, 128, 128, 3) - Y shape: (13711,)


In [4]:
# Phân chia dữ liệu thành tập huấn luyện và tập kiểm tra
x1_train, x1_test, y1_train, y1_test = train_test_split(X1, Y1, test_size=0.3, random_state=42)
x2_train, x2_test, y2_train, y2_test = train_test_split(X2, Y2, test_size=0.3, random_state=42)
x3_train, x3_test, y3_train, y3_test = train_test_split(X3, Y3, test_size=0.3, random_state=42)
x4_train, x4_test, y4_train, y4_test = train_test_split(X4, Y4, test_size=0.3, random_state=42)

x_train = np.concatenate((x1_train, x2_train, x3_train, x4_train), axis=0)
x_test = np.concatenate((x1_test, x2_test, x3_test, x4_test), axis=0)
y_train = np.concatenate((y1_train, y2_train, y3_train, y4_train), axis=0)
y_test = np.concatenate((y1_test, y2_test, y3_test, y4_test), axis=0)

print(f'x_train shape: {x_train.shape} - y_train shape: {y_train.shape}')
print(f'x_test shape: {x_test.shape} - y_test shape: {y_test.shape}')

x_train shape: (17582, 128, 128, 3) - y_train shape: (17582,)
x_test shape: (7537, 128, 128, 3) - y_test shape: (7537,)


In [5]:
# Đếm số lượng mẫu cho mỗi nhóm label trong tập train
unique_train, counts_train = np.unique(y_train, return_counts=True)
train_label_counts = dict(zip(unique_train, counts_train))
print("Train Label Counts:", train_label_counts)

# Đếm số lượng mẫu cho mỗi nhóm label trong tập test
unique_test, counts_test = np.unique(y_test, return_counts=True)
test_label_counts = dict(zip(unique_test, counts_test))
print("Test Label Counts:", test_label_counts)

Train Label Counts: {1: 734, 2: 5039, 3: 2212, 4: 9597}
Test Label Counts: {1: 315, 2: 2160, 3: 948, 4: 4114}


In [6]:
# Lưu dữ liệu vào các tập tin .npz
np.savez('/kaggle/working/training_data.npz', x_train=x_train, y_train=y_train)
np.savez('/kaggle/working/testing_data.npz', x_test=x_test, y_test=y_test)

# In thông báo sau khi lưu thành công
print("Dữ liệu đã được lưu thành công.")

Dữ liệu đã được lưu thành công.


In [7]:
# # Đọc dữ liệu từ tập tin .npz
# loaded_data_train = np.load('/kaggle/working/training_data.npz')
# loaded_data_test = np.load('/kaggle/working/testing_data.npz')

# # Truy cập mảng từ dữ liệu đã đọc
# x_train = loaded_data_train['x_train']
# y_train = loaded_data_train['y_train']
# x_test = loaded_data_test['x_test']
# y_test = loaded_data_test['y_test']

# # In thông báo sau khi tải thành công
# print("Dữ liệu đã được tải thành công.")

# # Kiểm tra kích thước của dữ liệu
# print(f'x_train shape: {x_train.shape} - y_train shape: {y_train.shape}')
# print(f'x_test shape: {x_test.shape} - y_test shape: {y_test.shape}')